# **\[FRAUD\]** 데이터정리 시도(9.20_df50 autogluon)

김보람  
2023-09-20

# imports

In [66]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import networkx as nx
import sklearn
import xgboost as xgb

# sklearn
from sklearn import model_selection # split함수이용
from sklearn import ensemble # RF,GBM
from sklearn import metrics
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB

# gnn
import torch
import torch.nn.functional as F
import torch_geometric
from torch_geometric.nn import GCNConv

# autogluon
from autogluon.tabular import TabularDataset, TabularPredictor

In [4]:
def down_sample_textbook(df):
    df_majority = df[df.is_fraud==0].copy()
    df_minority = df[df.is_fraud==1].copy()
    df_maj_dowsampled = sklearn.utils.resample(df_majority, n_samples=len(df_minority), replace=False, random_state=42)
    df_downsampled = pd.concat([df_minority, df_maj_dowsampled])
    return df_downsampled

def compute_time_difference(group):
    n = len(group)
    result = []
    for i in range(n):
        for j in range(n):
            time_difference = abs(group.iloc[i].trans_date_trans_time.value - group.iloc[j].trans_date_trans_time.value)
            result.append([group.iloc[i].name, group.iloc[j].name, time_difference])
    return result


class GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(1, 16)
        self.conv2 = GCNConv(16,2)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)

In [5]:
fraudTrain = pd.read_csv("~/Desktop/fraudTrain.csv").iloc[:,1:]

In [6]:
fraudTrain = fraudTrain.assign(trans_date_trans_time= list(map(lambda x: pd.to_datetime(x), fraudTrain.trans_date_trans_time)))
fraudTrain

## 데이터정리

In [40]:
_df1 = fraudTrain[fraudTrain["is_fraud"] == 0].sample(frac=0.20, random_state=42)
_df2 = fraudTrain[fraudTrain["is_fraud"] == 1]
df02 = pd.concat([_df1,_df2])
df02.shape

In [41]:
df50 = down_sample_textbook(df02)
df50.shape

In [42]:
df50 = df50.reset_index()

In [43]:
N = len(df50)

In [47]:
df50 = df50[["amt","is_fraud"]]

In [57]:
df50["amt"].mean()

In [58]:
df50["amt"].describe()

------------------------------------------------------------------------

### tr/test

In [48]:
df50_tr,df50_test = sklearn.model_selection.train_test_split(df50, random_state=42)

In [49]:
df50_tr.shape, df50_test.shape

In [50]:
train_mask = [i in df50_tr.index for i in range(N)]
test_mask = [i in df50_test.index for i in range(N)]

In [51]:
train_mask = np.array(train_mask)
test_mask = np.array(test_mask)

In [52]:
train_mask.sum(), test_mask.sum()

In [53]:
train_mask.shape, test_mask.shape

------------------------------------------------------------------------

### edge_index 설정

In [17]:
# groups = df50.groupby('cc_num')
# edge_index_list_plus = [compute_time_difference(group) for _, group in groups]
# edge_index_list_plus_flat = [item for sublist in edge_index_list_plus for item in sublist]
# edge_index_list_plus_nparr = np.array(edge_index_list_plus_flat)
# np.save('edge_index_list_plus50.npy', edge_index_list_plus_nparr)

In [78]:
edge_index = np.load('edge_index_list_plus50.npy')
theta = edge_index[:,2].mean()
edge_index = np.load('edge_index_list_plus50.npy').astype(np.float64)
edge_index[:,2] = (np.exp(-edge_index[:,2]/theta) != 1)*(np.exp(-edge_index[:,2]/theta))
edge_index = edge_index.tolist()
mean_ = np.array(edge_index)[:,2].mean()
selected_edges = [(int(row[0]), int(row[1])) for row in edge_index if row[2] > mean_]
edge_index_selected = torch.tensor(selected_edges, dtype=torch.long).t()

------------------------------------------------------------------------

### data설정(x, edge_index, y)

In [79]:
x = torch.tensor(df50['amt'], dtype=torch.float).reshape(-1,1)
y = torch.tensor(df50['is_fraud'],dtype=torch.int64)
data = torch_geometric.data.Data(x=x, edge_index = edge_index_selected, y=y, train_mask = train_mask, test_mask = test_mask)
data

------------------------------------------------------------------------

# autogluon

## A. 데이터

In [54]:
tr = TabularDataset(df50_tr)
tst = TabularDataset(df50_test)

## B. predictor 생성

In [55]:
predictr = TabularPredictor("is_fraud")

No path specified. Models will be saved in: "AutogluonModels/ag-20230920_153744/"

## C.적합(fit)

In [56]:
predictr.fit(tr) 

Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels/ag-20230920_153744/"
AutoGluon Version:  0.8.2
Python Version:     3.8.18
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #26~22.04.1-Ubuntu SMP PREEMPT_DYNAMIC Thu Jul 13 16:27:29 UTC 2
Disk Space Avail:   774.66 GB / 982.82 GB (78.8%)
Train Data Rows:    9009
Train Data Columns: 1
Label Column: is_fraud
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
    2 unique label values:  [1, 0]
    If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
    Available Memory:                    34935.91 MB
    Train Data (

In [59]:
predictr.leaderboard()

                  model  score_val  pred_time_val  fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0   WeightedEnsemble_L2   0.902331       0.051067  7.458454                0.001309           0.513563            2       True         14
1         LightGBMLarge   0.894562       0.001894  0.414943                0.001894           0.414943            1       True         13
2              CatBoost   0.894562       0.001981  0.653966                0.001981           0.653966            1       True          7
3               XGBoost   0.893452       0.003316  0.114061                0.003316           0.114061            1       True         11
4              LightGBM   0.892342       0.003488  0.343734                0.003488           0.343734            1       True          4
5        NeuralNetTorch   0.885683       0.005610  5.186066                0.005610           5.186066            1       True         12
6        KNeighborsUnif   0.877913

## D. 예측(predict)

In [60]:
(tr.is_fraud == predictr.predict(tr)).mean()

In [80]:
(tst.is_fraud == predictr.predict(tst)).mean()

In [61]:
yyhat = predictr.predict(tr)

> autogluon이렇게 하는게 맞는감…;;;